In [ ]:
import sqlite3

# Add a timeout to handle locking issues
conn = sqlite3.connect('trackdb.sqlite', timeout=10)
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
            DROP TABLE IF EXISTS Artist;
            DROP TABLE IF EXISTS Album;
            DROP TABLE IF EXISTS Track;
            DROP TABLE IF EXISTS Genre;

            CREATE TABLE Artist (
                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                name    TEXT UNIQUE
            );

            CREATE TABLE Album (
                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                artist_id INTEGER,
                title   TEXT UNIQUE,
                FOREIGN KEY (artist_id) REFERENCES Artist(id)
            );
            
             CREATE TABLE Genre (
                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                name   TEXT UNIQUE
                
            );
            
             CREATE TABLE Track (
                id  INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT  UNIQUE,
                genre_id INTEGER,
                album_id  INTEGER,
                len INTEGER, rating INTEGER, count INTEGER,
                FOREIGN KEY (genre_id) REFERENCES Genre(id), 
                FOREIGN KEY (album_id) REFERENCES Album(id)
            );

            ''')

# Prompt the user for the file name
ftracks = input('Enter file name: ') #use [tracks.csv] 
if len(ftracks) < 1:
    ftracks = 'tracks.csv'

# Try opening the file with error handling
try:
    ftracks = open(ftracks,  encoding='ISO-8859-1')
except FileNotFoundError:
    print(f"Error: File '{ftracks}' not found.")
    exit()

# Process each line from the file
for line in ftracks:
    line = line.strip()
    pieces = line.split(',')
    if len(pieces) < 6: 
        continue

    name = pieces[0]
    artist = pieces[1]
    album = pieces[2]
    count = pieces[3]
    rating = pieces[4]
    length = pieces[5]
    genre = pieces[6] if len(pieces) > 6 else None  # Handle missing genre

    # print(name, artist, album, count, rating, length, genre)

    # Insert or ignore Artist
    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', (artist,))
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist,))
    artist_id = cur.fetchone()[0]

    # Insert or ignore Album
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', (album, artist_id))
    
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album,))
    album_id = cur.fetchone()[0]
    
    #insert or ignore Genre
    if genre:
        cur.execute('''INSERT OR IGNORE INTO Genre (name) 
                VALUES ( ? )''', (genre,))
        cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre,))
        genre_id = cur.fetchone()[0]
    else:
        genre_id = None

    # Insert or replace Track
    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, rating, count, genre_id) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        (name, album_id, length, rating, count, genre_id))
    
    
    # Commit the changes once, after all rows have been processed
conn.commit()

 #Query the database to select the top 10 emails with the highest counts    
sqlstr = ''' 
SELECT Album.title, Artist.name
FROM Album
JOIN Artist ON Album.artist_id = Artist.id
ORDER BY Artist.name DESC
LIMIT 10; '''


# Loop through the rows returned by the SQL query and print each album and its artist
for row in cur.execute(sqlstr):
    print(f"Album: {row[0]}, Artist: {row[1]}")
# Close the database connection after the script finishes
conn.close()
